# PART 1

### Import data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_data = os.path.join(processed_data_path, 'train.csv')
test_data = os.path.join(processed_data_path, 'test.csv')

In [3]:
train_data = pd.read_csv(train_data, index_col = 'PassengerId')
test_data = pd.read_csv(test_data, index_col = 'PassengerId')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null int64
Deck_B                891 non-null int64
Deck_C                891 non-null int64
Deck_D                891 non-null int64
Deck_E                891 non-null int64
Deck_F                891 non-null int64
Deck_G                891 non-null int64
Deck_Z                891 non-null int64
Pclass_1              891 non-null int64
Pclass_2              891 non-null int64
Pclass_3              891 non-null int64
Title_Lady            891 non-null int64
Title_Master          891 non-null int64
Title_Miss            891 non-null int64
Title_Mr              891 non-null int64
Title_Mrs             891 non-

In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null int64
Deck_B                418 non-null int64
Deck_C                418 non-null int64
Deck_D                418 non-null int64
Deck_E                418 non-null int64
Deck_F                418 non-null int64
Deck_G                418 non-null int64
Deck_Z                418 non-null int64
Pclass_1              418 non-null int64
Pclass_2              418 non-null int64
Pclass_3              418 non-null int64
Title_Lady            418 non-null int64
Title_Master          418 non-null int64
Title_Miss            418 non-null int64
Title_Mr              418 non-null int64
Title_Mrs             418 non-null int64
Title_Officer         418 n

### Data Preparation

In [19]:
#Changes all the columns except the survival column to a matrix using .values method
X = train_data.loc[:, 'Age':].values.astype('float')

#Changes the row into an one dimensional array, aka vector
y = train_data['Survived'].ravel()                       

In [20]:
print (X.shape, y.shape)

(891, 32) (891,)


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
#Printing train dimensions.
print(X_train.shape, y_train.shape)

(712, 32) (712,)


In [29]:
#printing test dimensions.
print(X_test.shape, y_test.shape)

(179, 32) (179,)


In [39]:
#Average survival in test and train data
print('Averge survived in train: {0:.3f}'.format(np.mean(y_train)))
print('Averge survived in test: {0:.3f}'.format(np.mean(y_test)))

Averge survived in train: 0.383
Averge survived in test: 0.385


#### Checking sklearn version for creating the baseline model.

In [40]:
import sklearn

In [42]:
sklearn.__version__ ##v0.19 is necessary for using DummyClassifier() method.

'0.19.1'

## Creating the baseline model

In [44]:
from sklearn.dummy import DummyClassifier

In [69]:
# Here most frequent refers to the label that occurs most frequently, In our case 0, Which means we tell the model to predict all
# the input as zero
dummy_model = DummyClassifier(strategy = 'most_frequent', random_state = 0)

In [70]:
dummy_model.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [71]:
#This basically is the benchmark, that means, anything below or equal to the output accuracy would make no sense
print('Score/Accuracy of baseline model = {0:.2f}'.format(dummy_model.score(X_test, y_test)))

Score/Accuracy of baseline model = 0.61


In [72]:
#Getting all the performance metrics and the confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [73]:
print('Accuracy of baseline model = {0:.2f}'.format(accuracy_score(y_test, dummy_model.predict(X_test))))

Accuracy of baseline model = 0.61


In [74]:
# Since all the inputs are predicted 0, true positive is 0, which can be seen in the confusion matrix
print('Confusion Matrix = \n {0}'.format(confusion_matrix(y_test, dummy_model.predict(X_test))))

Confusion Matrix = 
 [[110   0]
 [ 69   0]]


In [60]:
#since true positive is negative, we can see that the precision and accuracy are both 0.
print('Precision of baseline model = {:.2f}'.format(precision_score(y_test, dummy_model.predict(X_test))))
print('Recall of baseline model = {:.2f}'.format(recall_score(y_test, dummy_model.predict(X_test))))

Precision of baseline model = 0.00
Recall of baseline model = 0.00


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### First kaggle submission

In [75]:
test_X = test_data.values.astype('float')

In [76]:
predictions = dummy_model.predict(test_X)

In [80]:
submission_data = pd.DataFrame({'PassengerId' : test_data.index, 'Survived':predictions})

In [83]:
submission_data.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [84]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, 'dummy_submission.csv')

In [86]:
submission_data.to_csv(submission_file_path, index = False)

In [89]:
def get_submission_file(model, filename):
    #Converting test data into a matrix 
    test_X = test_data.values.astype('float')
    
    #make predictions
    predictions = dummy_model.predict(test_X)
    
    submission_data = pd.DataFrame({'PassengerId' : test_data.index, 'Survived': predictions})
    
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, filename)
    
    submission_data.to_csv(submission_file_path, index = False)

In [90]:
get_submission_file(dummy_model, 'dummy_submission.csv')

### Predicting using Logistic Regression

In [91]:
from sklearn.linear_model import LogisticRegression

In [92]:
model_lr = LogisticRegression(random_state = 0)

In [93]:
model_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
print('Score for logistic regression: {:.2f}'.format(model_lr.score(X_test, y_test)))

Score for logistic regression: 0.83


In [102]:
print('Accuracy using logistic regression: {:.2f}'.format(accuracy_score(y_test, model_lr.predict(X_test))))

print('Confusion matrix using logistic regression: \n {0}'.format(confusion_matrix(y_test, model_lr.predict(X_test))))

print('Precision using logistic regression: {:.2f}'.format(precision_score(y_test, model_lr.predict(X_test))))

print('Recall using logistic regression: {:.2f}'.format(recall_score(y_test, model_lr.predict(X_test))))

Accuracy using logistic regression: 0.83
Confusion matrix using logistic regression: 
 [[95 15]
 [15 54]]
Precision using logistic regression: 0.78
Recall using logistic regression: 0.78


In [105]:
model_lr.coef_

array([[-0.02840734,  0.00455631, -0.50017004,  0.61922838, -0.81414743,
         0.12823264, -0.17253859, -0.39355488,  0.52215008,  1.09939125,
         0.40346551, -0.18369316, -0.30021028,  0.96558544,  0.48281794,
        -0.3451608 ,  0.28258585,  1.21850069,  0.56334183, -1.44612507,
         1.07146232, -0.11345497, -0.47306807,  0.16297326,  0.24746349,
         0.27998252,  0.4128233 ,  0.49202884,  0.46214499,  0.14906873,
         0.37253571,  0.73070686]])

#### Second Kaggle submission

In [106]:
get_submission_file(model_lr, 'model_lr.csv')

# PART 2

## Hyperparameter Optimzation

In [110]:
model_lr_2 = LogisticRegression(random_state = 0)

In [112]:
from sklearn.model_selection import GridSearchCV

In [144]:
parameters = {'C' : [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['l1', 'l2']}
clf = GridSearchCV(model_lr_2, param_grid = parameters, cv = 3, error_score = 0.0)

In [145]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=0.0,
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [147]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [148]:
print('The Score for losgistic regression with 3 fold cross validation: {:.2f}'.format(clf.score(X_test, y_test)))

The Score for losgistic regression with 3 fold cross validation: 0.83


####  Third kaggle submission

In [149]:
get_submission_file(clf, 'model_lr_2.csv')

## Feature Normalization and Standardization

In [151]:
#MinMaxScalar for normalization and StandardScaler for Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature Normalization

In [ ]:
normalize = MinMaxScaler()
X_train_scaled = normalize